In [1]:
import pandas as pd
import requests

def get_option_chain_data(instrument_name: str, expiry_date: str, side: str) -> pd.DataFrame:
    """
    Fetches option chain data for a given instrument and expiry date, returning the highest bid price for Put options
    or the highest ask price for Call options for each strike price.

    Parameters:
    - instrument_name (str): Name of the instrument (e.g., NIFTY or BANKNIFTY).
    - expiry_date (str): Expiration date in YYYY-MM-DD format.
    - side (str): Type of option to retrieve, either "PE" for Put or "CE" for Call.

    Returns:
    - pd.DataFrame: DataFrame with columns: instrument_name, strike_price, side, and bid/ask.
    """
    # API endpoint and parameters
    url = "https://api.upstox.com/option-chain"  # Replace with actual API endpoint
    params = {
        "instrument": instrument_name,
        "expiry": expiry_date
    }
    headers = {
        "Authorization": "Bearer YOUR_API_KEY"  # Replace with your actual API key
    }

    # Make the API request
    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    # Initialize list to store the data
    option_data = []

    # Process each option contract
    for option in data.get('options', []):
        strike_price = option['strike_price']
        if side == "PE" and option['type'] == "PE":
            bid_price = option['bid_price']
            option_data.append([instrument_name, strike_price, side, bid_price])
        elif side == "CE" and option['type'] == "CE":
            ask_price = option['ask_price']
            option_data.append([instrument_name, strike_price, side, ask_price])

    # Convert the data to a DataFrame
    df = pd.DataFrame(option_data, columns=["instrument_name", "strike_price", "side", "bid/ask"])
    return df


In [2]:
def calculate_margin_and_premium(data: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates margin and premium earned for each option in the provided DataFrame.

    Parameters:
    - data (pd.DataFrame): DataFrame from get_option_chain_data.

    Returns:
    - pd.DataFrame: Modified DataFrame with additional columns: margin_required and premium_earned.
    """
    # API endpoint for margin calculation
    margin_url = "https://api.upstox.com/margin-requirement"  # Replace with actual endpoint
    headers = {
        "Authorization": "Bearer YOUR_API_KEY"  # Replace with your actual API key
    }

    # List to store the results
    results = []

    for _, row in data.iterrows():
        instrument_name = row["instrument_name"]
        strike_price = row["strike_price"]
        bid_ask_price = row["bid/ask"]

        # Request margin requirement for each option
        params = {
            "instrument": instrument_name,
            "strike_price": strike_price,
            "transaction_type": "Sell"  # Transaction type as "Sell"
        }
        response = requests.get(margin_url, headers=headers, params=params)
        margin_data = response.json()
        margin_required = margin_data.get('margin', 0)  # Default to 0 if margin data is missing

        # Calculate premium earned (bid/ask * lot size)
        lot_size = 120  # Example lot size; replace with dynamic data if available
        premium_earned = bid_ask_price * lot_size

        # Append results
        results.append([
            instrument_name, strike_price, row["side"], bid_ask_price, margin_required, premium_earned
        ])

    # Convert to DataFrame
    df_result = pd.DataFrame(results, columns=[
        "instrument_name", "strike_price", "side", "bid/ask", "margin_required", "premium_earned"
    ])
    return df_result


In [3]:
# Part 1: Retrieve option chain data
option_data = get_option_chain_data("NIFTY", "2024-11-30", "PE")

# Part 2: Calculate margin and premium
final_data = calculate_margin_and_premium(option_data)

# Display the resulting DataFrame
print(final_data)


Empty DataFrame
Columns: [instrument_name, strike_price, side, bid/ask, margin_required, premium_earned]
Index: []
